<a href="https://colab.research.google.com/github/Otter2022/RowdyDatathon24/blob/main/PysparkAnlysisOfAQS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests pyspark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Path = ""

import os
import zipfile
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("PM2.5 Data Unzipping and Loading").getOrCreate()

# Directory containing the zip files
zip_dir = "/content/drive/MyDrive/Datathon24/PM2.5data"
extracted_dir = "/content/drive/MyDrive/Datathon24/PM2.5dataunzipped"

# Function to unzip files
def unzip_files(zip_dir, extracted_dir):
    for filename in os.listdir(zip_dir):
        if filename.endswith(".zip"):
            zip_path = os.path.join(zip_dir, filename)
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extracted_dir)
                print(f"Extracted: {filename}")

# Unzip all files in the directory
unzip_files(zip_dir, extracted_dir)

# Show the files extracted
print("Extracted files:", os.listdir(extracted_dir))


Extracted: hourly_88101_2015.zip
Extracted: hourly_88101_2014.zip
Extracted: hourly_88101_2018.zip
Extracted: hourly_88101_2017.zip
Extracted: hourly_88101_2016.zip
Extracted: hourly_88101_2022.zip
Extracted: hourly_88101_2021.zip
Extracted: hourly_88101_2020.zip
Extracted: hourly_88101_2019.zip
Extracted: hourly_88101_2024.zip
Extracted: hourly_88101_2023.zip
Extracted files: ['hourly_88101_2015.csv', 'hourly_88101_2014.csv', 'hourly_88101_2018.csv', 'hourly_88101_2017.csv', 'hourly_88101_2016.csv', 'hourly_88101_2022.csv', 'hourly_88101_2021.csv', 'hourly_88101_2020.csv', 'hourly_88101_2019.csv', 'hourly_88101_2024.csv', 'hourly_88101_2023.csv']


In [ ]:
# Load multiple CSV files into a single PySpark DataFrame
def load_csv_files(extracted_dir):
    csv_files = [os.path.join(extracted_dir, f) for f in os.listdir(extracted_dir) if f.endswith(".csv")]

    # If there are multiple CSVs, you can load them all into one DataFrame
    combined_df = None
    for csv_file in csv_files:
        df = spark.read.option("header", "true").csv(csv_file)
        if combined_df is None:
            combined_df = df
        else:
            combined_df = combined_df.union(df)

    return combined_df

# Load all CSV files into a single DataFrame
pm25_df = load_csv_files(extracted_dir)

# Show the first few rows
pm25_df.show(10)


+----------+-----------+--------+--------------+---+---------+---------+-----+--------------------+----------+----------+----------+--------+------------------+--------------------+---+-----------+---------+-----------+-----------+--------------------+----------+-----------+-------------------+
|State Code|County Code|Site Num|Parameter Code|POC| Latitude|Longitude|Datum|      Parameter Name|Date Local|Time Local|  Date GMT|Time GMT|Sample Measurement|    Units of Measure|MDL|Uncertainty|Qualifier|Method Type|Method Code|         Method Name|State Name|County Name|Date of Last Change|
+----------+-----------+--------+--------------+---+---------+---------+-----+--------------------+----------+----------+----------+--------+------------------+--------------------+---+-----------+---------+-----------+-----------+--------------------+----------+-----------+-------------------+
|        01|        073|    0023|         88101|  3|33.553056|  -86.815|WGS84|PM2.5 - Local Con...|2015-01-01|  

In [ ]:
# Save the DataFrame as a Parquet file
pm25_df.write.mode("overwrite").parquet("/content/drive/MyDrive/Datathon24/PM2.5_2014_2024")
